# Install enviroment

In [ ]:
# # install dependencies: (use cu111 because colab has CUDA 11.1)
# !pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# # install MMCV
# !pip install mmcv-full==1.6.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

# # install MMDetection
# !pip install mmdet==2.28.1

# # Install mmdetection
# %cd drive/MyDrive/mmtracking

# # clone the MMTracking repository
# # !git clone https://github.com/open-mmlab/mmtracking.git

# # install MMTracking and its dependencies
# !pip install -q -r requirements/build.txt
# !pip install -q -e .

# # used to MOT evaluation
# !pip install -q git+https://github.com/JonathonLuiten/TrackEval.git

In [1]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

# install MMCV
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html

# install MMDetection
!pip install mmdet==2.28.1

# clone the MMTracking repository
!git clone https://github.com/open-mmlab/mmtracking.git

# Install mmdetection
%cd ./mmtracking

# install MMTracking and its dependencies
!pip install -q -r requirements/build.txt
!pip install -q -e .

# used to MOT evaluation
!pip install -q git+https://github.com/JonathonLuiten/TrackEval.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 31.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.2+cu118
    Uninstalling torchaudio-2.0.2+cu118:
      Successfully uninstalled torchaudio-2.0.2+cu118
ERROR: pip's dependency resolver does not currently take into account all the 

In [2]:
import mmcv
import tempfile
from mmtrack.apis import inference_mot, init_model
import cv2
from collections import Counter
import logging
from datetime import datetime, timedelta
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd

/usr/local/lib/python3.10/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [3]:
%cd ../drive/MyDrive/mmtracking

/content/drive/MyDrive/mmtracking


# Run Trackor

## Develop id collection function

In [4]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxBArea)
	# return the intersection over union value
	return iou

def collect_object_id(obj_list,tracking,img,start_point,end_point):
  boxA = start_point+end_point
  for cls, objs in enumerate(tracking):
    for obj in objs:
      id = int(obj[0])
      conf = obj[5]
      boxB = [obj[1],obj[2],obj[3],obj[4]]
      iou = bb_intersection_over_union(boxA, boxB)
      if iou>0.5:
        if id in list(obj_list.keys()):
          if conf > obj_list.get(id)[1]:
            obj_list[id] = [cls,conf]
        else:
          obj_list[id] = [cls,conf]
  return obj_list

def roi_cornner(img, axis, start, length):
  y_shape, x_shape = img.shape[0], img.shape[1]
  if axis==0:
    start_point = (start,0)
    end_point = (start+length,y_shape)
  elif axis==1:
    start_point = (0,start)
    end_point = (x_shape,start+length)
  else:
    start_point = (0,0)
    end_point = (x_shape,y_shape)

  return start_point[0], start_point[1], end_point[0], end_point[1]

def drawROI(img, start_point, end_point):
  img = cv2.rectangle(img, start_point, end_point, (0,255,0),5)
  return img

1. obj_score_thrs (dict): Detection score threshold for matching objects.
*   high (float): Threshold of the first matching. Defaults to 0.6.
*   low (float): Threshold of the second matching. Defaults to 0.1.

2. init_track_thr (float): Detection score threshold for initializing a new tracklet. Defaults to 0.7.

3. weight_iou_with_det_scores (bool): Whether using detection scores to weight IOU which is used for matching. Defaults to True.

4. match_iou_thrs (dict): IOU distance threshold for matching between two frames.
* high (float): Threshold of the first matching. Defaults to 0.1.
* low (float): Threshold of the second matching. Defaults to 0.5.

5. tentative (float): Threshold of the matching for tentative tracklets. Defaults to 0.3.

6. num_tentatives (int, optional): Number of continuous frames to confirm a track. Defaults to 3.

In [5]:
# --------------- Sideview setting ---------------
# checkpoint = 'yolox_s_openimage+sideview_20220729_epoch_132'
# checkpoint = 'yolox_l_sideview_epoch_120'
checkpoint = 'yolox_x_sideview_epoch_84'
# video = '1'
# video = '2'
video = '3'
# video_type = 'mov'
video_type = 'mp4'
# axis = 0 # with ROI
axis = 2 # without ROI
roi_start,roi_length = 150,300 # video size 640x640
# roi_start,roi_length = 300,600 # video size 1280x720
img_scale = (640, 640)
crop = 0

# --------------- Topview setting ---------------
# checkpoint = 'yolox_s_topview_20220728_epoch_108'
# checkpoint = 'yolox_l_topview_epoch_96'
# checkpoint = 'yolox_x_topview_epoch_108'
# video = '1minute_topview'
# video_type = 'mp4'
# axis = 1
# # axis = 2 # without ROI
# roi_start = 300
# roi_length = 200
# img_scale = (640, 640)
# crop = 1

# --------------- config file ---------------
# mot_config = 'configs/mot/bytetrack/bytetrack_yolox_s_sideview.py'
# mot_config = 'configs/mot/bytetrack/bytetrack_yolox_l.py'
mot_config = 'configs/mot/bytetrack/bytetrack_yolox_x.py'

# set config file path
input_video = f'./demo/{video}.{video_type}'

# --------------- Set option ---------------
cfg = dict(
    model=dict(
        detector=dict(
            input_size=img_scale,
            random_size_range=(18, 32), # (18,32), (15, 25)
            test_cfg=dict(
                score_thr=0.4
                , nms=dict(type='nms', iou_threshold=0.1)
                ),
            init_cfg=dict(
                checkpoint=f'/content/drive/MyDrive/mmtracking/checkpoints/{checkpoint}.pth'
            )
        ),
        motion = dict(type='KalmanFilter'),
        tracker = dict(
            type='ByteTracker',
            obj_score_thrs=dict(high=0.6, low=0.1),
            init_track_thr=0.7,
            weight_iou_with_det_scores=True,
            match_iou_thrs=dict(high=0.1, low=0.5, tentative=0.3),
            num_frames_retain=90,
            num_tentatives=1
        )
    ),
    # test_pipeline = [
    #     dict(
    #         img_scale=img_scale
    #     )
    # ]
)

# run mot
imgs = mmcv.VideoReader(input_video)

# set collected object list
obj_list = dict()

# build the model from a config file
mot_model = init_model(mot_config
                       , device='cuda:0'
                       , checkpoint=f'./checkpoints/{checkpoint}.pth'
                       , cfg_options=cfg
                       )
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name

count_miss = 0

# test and show/save the images
for i, img in enumerate(imgs):

  if video =='1minute_topview':
    img = img[80:,640:] # crop img

  img = cv2.resize(img,(640,640))

  # Run trackor
  result = inference_mot(mot_model, img, frame_id=i)

  # Calculation ROI corner
  start_roi_x, start_roi_y, end_roi_x, end_roi_y = roi_cornner(img=img, axis=axis, start=roi_start, length=roi_length)

  # Draw ROI area
  img = drawROI(img=img, start_point=(start_roi_x, start_roi_y), end_point=(end_roi_x, end_roi_y))

  obj_list = collect_object_id(obj_list
                               ,result.get('track_bboxes')
                               , img=img
                               , start_point=[start_roi_x,start_roi_y]
                               , end_point=[end_roi_x,end_roi_y])

  mot_model.show_result(
            img,
            result,
            show=False,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')

  prog_bar.update()

dt = datetime.now() + timedelta(hours=7)

# Export tracking video
timestamp = dt.strftime('%H:%M_(%d-%m-%Y)')
output = f'./demo/video/byteTrack_{video}_{checkpoint}_{timestamp}.mp4'
print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2023-06-23 05:06:48,358 - mmcv - INFO - initialize YOLOX with init_cfg {'type': 'Pretrained', 'checkpoint': '/content/drive/MyDrive/mmtracking/checkpoints/yolox_x_sideview_epoch_84.pth'}
2023-06-23 05:06:48,360 - mmcv - INFO - load model from: /content/drive/MyDrive/mmtracking/checkpoints/yolox_x_sideview_epoch_84.pth
2023-06-23 05:06:48,366 - mmcv - INFO - load checkpoint from local path: /content/drive/MyDrive/mmtracking/checkpoints/yolox_x_sideview_epoch_84.pth
2023-06-23 05:07:19,316 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_backbone_stem_conv_conv_weight, ema_backbone_stem_conv_bn_weight, ema_backbone_stem_conv_bn_bias, ema_backbone_stem_conv_bn_running_mean, ema_backbone_stem_conv_bn_running_var, ema_backbone_stem_conv_bn_num_batches_tracked, ema_backbone_stage1_0_conv_weight, ema_backbone_stage1_0_bn_weight, ema_backbone_stage1_0_bn_bias, ema_backbone_stage1_0_bn_running_mean, ema_backbone_stage1_0_bn_runnin

load checkpoint from local path: ./checkpoints/yolox_x_sideview_epoch_84.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_backbone_stem_conv_conv_weight, ema_backbone_stem_conv_bn_weight, ema_backbone_stem_conv_bn_bias, ema_backbone_stem_conv_bn_running_mean, ema_backbone_stem_conv_bn_running_var, ema_backbone_stem_conv_bn_num_batches_tracked, ema_backbone_stage1_0_conv_weight, ema_backbone_stage1_0_bn_weight, ema_backbone_stage1_0_bn_bias, ema_backbone_stage1_0_bn_running_mean, ema_backbone_stage1_0_bn_running_var, ema_backbone_stage1_0_bn_num_batches_tracked, ema_backbone_stage1_1_main_conv_conv_weight, ema_backbone_stage1_1_main_conv_bn_weight, ema_backbone_stage1_1_main_conv_bn_bias, ema_backbone_stage1_1_main_conv_bn_running_mean, ema_backbone_stage1_1_main_conv_bn_running_var, ema_backbone_stage1_1_main_conv_bn_num_batches_tracked, ema_backbone_stage1_1_short_conv_conv_weight, ema_backbone_stage1_1_short_conv_bn_weight, ema_backbo

/usr/local/lib/python3.10/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1904/1904, 8.4 task/s, elapsed: 225s, ETA:     0s
 making the output video at ./demo/video/byteTrack_3_yolox_x_sideview_epoch_84_12:11_(23-06-2023).mp4 with a FPS of 30.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1904/1904, 67.4 task/s, elapsed: 28s, ETA:     0s


In [ ]:
dt = datetime.now() + timedelta(hours=7)

LOG_FILENAME = dt.strftime('log/logfile_%H:%M_(%d-%m-%Y)')
LOG_FILENAME = LOG_FILENAME + f'_{video}.log'
for handler in logging.root.handlers[:]:
  logging.root.removeHandler(handler)

logging.basicConfig(filename=LOG_FILENAME,level=logging.INFO)

vehicle = ['Bus','Car','Motorcycle','Pickup','Truck','Van']
vehicles = []
for key in obj_list:
  cls_ = obj_list[key][0]
  cls_name = vehicle[cls_]
  logging.info(f'{key}>>{cls_name}')
  vehicles.append(cls_name)
vehicle_count = dict(Counter(vehicles))

logging.info('-------- Vehicle counting --------')

for class_ in ['Bus','Car','Motorcycle','Pickup','Truck','Van']:
  print(f'{class_}>>{vehicle_count.get(class_)}')
  logging.info(f'{class_}>>{vehicle_count.get(class_)}')

logging.shutdown()

Bus>>3
Car>>22
Motorcycle>>23
Pickup>>8
Truck>>2
Van>>1


## Grid search on ByteTrack

In [ ]:
dt = datetime.now() + timedelta(hours=7)

video = '2'
video_type = 'mov'
axis = 0
roi_start = 150
roi_length = 300
img_scale = (640, 640)
checkpoint = 'yolox_x_sideview_epoch_84'
gt = dict(
    Bus=1,
    Car=19,
    Motorcycle=22,
    Pickup=3,
    Van=3
)

# video = '1minute_topview'
# video_type = 'mp4'
# axis = 1
# roi_start = 300
# roi_length = 200
# img_scale = (640, 640)
# checkpoint = 'yolox_x_topview_epoch_108'
# gt = dict(
#     Bus=2,
#     Car=19,
#     Motorcycle=18,
#     Pickup=6,
#     Truck=2
# )


mot_config = 'configs/mot/bytetrack/bytetrack_yolox_x.py'

input_video = f'./demo/{video}.{video_type}'
imgs = mmcv.VideoReader(input_video)

grid_result = []
n_cond = 0

LOG_FILENAME = dt.strftime('./demo/gridSearch_%d-%m-%Y_(%H:%M)')
LOG_FILENAME = LOG_FILENAME + f'_{video}.csv'

def mape(gt, counting):
  error = 0
  total = 0
  for cls in ['Bus','Car','Motorcycle','Pickup','Truck','Van']:

    if gt.get(cls):
      total += gt.get(cls)
      error += abs(gt.get(cls)-counting.get(cls))

  return error/total

for score in [0.1,0.4]:
  for iou in [0.7,0.2]:
    for init_track in [0.7]:
      for match_iou_h in [0.1]:
        for match_iou_l in [0.5]:
          for num_frames in [30,90]:
            for num_tentatives in [3,1]:
              n_cond+=1
              cfg = dict(
                  model=dict(
                      detector=dict(
                          test_cfg=dict(
                              score_thr=score
                              , nms=dict(type='nms', iou_threshold=iou)
                              ),
                          init_cfg=dict(
                              checkpoint=f'/content/drive/MyDrive/mmtracking/checkpoints/{checkpoint}.pth'
                          )
                      ),
                      motion = dict(type='KalmanFilter'),
                      tracker = dict(
                          type='ByteTracker',
                          obj_score_thrs=dict(high=0.6, low=0.1),
                          init_track_thr=init_track,
                          weight_iou_with_det_scores=True,
                          match_iou_thrs=dict(high=match_iou_h, low=match_iou_l, tentative=0.3),
                          num_frames_retain=num_frames,
                          num_tentatives=num_tentatives
                      )
                  )
              )

              print(f'Run grid search on condiiton {n_cond}')
              obj_list = dict()
              mot_model = init_model(mot_config, device='cuda:0', cfg_options=cfg)
              prog_bar = mmcv.ProgressBar(len(imgs))
              out_dir = tempfile.TemporaryDirectory()
              out_path = out_dir.name

              # test and show/save the images
              for i, img in enumerate(imgs):

                # Crop Topview video
                if video =='1minute_topview':
                  img = img[80:,640:] # crop img

                # Run trackor
                img = cv2.resize(img,(640,640))
                result = inference_mot(mot_model, img, frame_id=i)

                # Calculation ROI corner
                start_roi_x, start_roi_y, end_roi_x, end_roi_y = roi_cornner(img=img, axis=axis, start=roi_start, length=roi_length)

                # Draw ROI area
                img = drawROI(img=img, start_point=(start_roi_x, start_roi_y), end_point=(end_roi_x, end_roi_y))

                obj_list = collect_object_id(obj_list
                                            ,result.get('track_bboxes')
                                            , img=img
                                            , start_point=[start_roi_x,start_roi_y]
                                            , end_point=[end_roi_x,end_roi_y])

                mot_model.show_result(
                          img,
                          result,
                          show=False,
                          wait_time=int(1000. / imgs.fps),
                          out_file=f'{out_path}/{i:06d}.jpg')

                prog_bar.update()

              out_dir.cleanup()

              print(f'Collect result on condiiton {n_cond}')

              vehicle = ['Bus','Car','Motorcycle','Pickup','Truck','Van']
              vehicles = []
              for key in obj_list:
                cls_ = obj_list[key][0]
                cls_name = vehicle[cls_]
                logging.info(f'{key}>>{cls_name}')
                vehicles.append(cls_name)
              vehicle_count = dict(Counter(vehicles))

              error = mape(gt, vehicle_count)

              grid_result.append(
                  dict(
                      # weight_iou_para=weight_iou,
                      score_threshold=score,
                      nms_iou=iou,
                      init_track_para=init_track,
                      match_iou_high_para=match_iou_h,
                      match_iou_low_para=match_iou_l,
                      num_frames_para=num_frames,
                      num_tentatives_para=num_tentatives,
                      Bus=vehicle_count.get('Bus'),
                      Car=vehicle_count.get('Car'),
                      Motorcycle=vehicle_count.get('Motorcycle'),
                      Pickup=vehicle_count.get('Pickup'),
                      Truck=vehicle_count.get('Truck'),
                      Van=vehicle_count.get('Van'),
                      MAPE=error
                  )
              )
              pd.DataFrame(grid_result).to_csv(LOG_FILENAME, index=False)

Run grid search on condiiton 1


2023-05-07 04:22:49,297 - mmtrack - INFO - initialize YOLOX with init_cfg {'type': 'Pretrained', 'checkpoint': '/content/drive/MyDrive/mmtracking/checkpoints/yolox_x_sideview_epoch_84.pth'}
2023-05-07 04:22:49,300 - mmcv - INFO - load model from: /content/drive/MyDrive/mmtracking/checkpoints/yolox_x_sideview_epoch_84.pth
2023-05-07 04:22:49,303 - mmcv - INFO - load checkpoint from local path: /content/drive/MyDrive/mmtracking/checkpoints/yolox_x_sideview_epoch_84.pth
